## Einleitung

In diesem Toy Projekt arbeiten wir mit Tweets von amerikanischen Politikern. Wir wollen einmal sehen, ob es möglich ist, den typischen Tweet-Style des amerikanischen Chefpolitikers mit Machine Learning zu erkennen und seine Tweets von anderen zu unterscheiden.

Dafür verwenden wir den Cloud-Dienst [Monkeylearn](https://monkeylearn.com). Monkeylearn hat sich auf Natural Language Processing spezialisiert, und bietet eine Vielzahl an vorkonfigurierten Klassifiern, die man mit eigenen Daten trainieren und verwenden kann.

Unsere Daten holen wir aus dem [Trump Twitter Archive](http://www.trumptwitterarchive.com/). Direkte Downloads von Archiven sind [hier](https://github.com/bpb27/trump-tweet-archive) und weitere Archive sind [hier](https://github.com/bpb27/political_twitter_archive).

In [ ]:
# Einige Archive sind bereits im Docker Image drin:
!ls /data/toyprojects/data/tweets/*

## Data Cleaning

Zuerst laden wir die Daten von Donald Trump:

In [ ]:
import json
with open('/data/toyprojects/data/tweets/donald_trump/condensed_2016.json', 'r') as f:
    trump_tweets = json.load(f)

Nun laden wir die Daten von Ben Carson, einem weiteren Republikaner. Wir wählen bewusst zwei Republikaner. Mit Trump und einem Demokraten wäre die Aufgabe wohl etwas zu einfach.

In [ ]:
with open('/data/toyprojects/data/tweets/others/realbencarson_short.json', 'r') as f:
    other_tweets = json.load(f)

Schauen wir uns die Struktur mal an:

In [ ]:
trump_tweets[0:2]

Es interessieren uns lediglich `text` und `is_retweet`. Wir säubern nun die Daten ein wenig. Zum Beispiel entfernen wir Links, @mentions, Hashtags. Dies wiederum, um dem Algorithmus die Sache nicht zu einfach zu machen.

In [ ]:
import re
def filter_and_clean_tweets(j):
    """Filter out retweets and clean remaining tweets
    downloaded from http://www.trumptwitterarchive.com/ a bit
    """
    
    # filter out retweets and extract tweet text
    tweets = [entry['text'] for entry in j if not entry['is_retweet']]

    # replace \n by space
    tweets = [re.sub(r'\n', ' ', tweet) for tweet in tweets]

    # remove double quotes
    tweets = [re.sub(r'"', '', tweet) for tweet in tweets]

    # remove leading dot
    tweets = [re.sub(r'^\.', '', tweet) for tweet in tweets]

    # remove 'RT ' at beginning
    tweets = [re.sub(r'^RT\s*', '', tweet) for tweet in tweets] 

    # remove @mentions
    tweets = [re.sub(r'@\w+:?', r'', tweet) for tweet in tweets] 

    # remove hashtags
    tweets = [re.sub(r'#\w+:?', r'', tweet) for tweet in tweets]

    # remove links (do it several times to catch them all)
    for i in range(3):
        tweets = [re.sub(r'(.*)\s*https?://.+\s*(.*)', r'\1 \2', tweet) for tweet in tweets]

    # remove whitespace from beginning and end
    tweets = [tweet.rstrip().lstrip() for tweet in tweets]

    # replace &amp; with &
    tweets = [re.sub(r'&amp;', r'&', tweet) for tweet in tweets]

    # condense multiple spaces
    tweets =[re.sub(r'\s+', r' ', tweet) for tweet in tweets]

    # return result for all tweets that are not empty now after the cleaning
    return [tweet for tweet in tweets if tweet != '']

In [ ]:
trump_tweets = filter_and_clean_tweets(trump_tweets)
other_tweets = filter_and_clean_tweets(other_tweets)

Schauen wir uns das schnell an (mit irgendwelchen zufälligen Indices)

In [ ]:
trump_tweets[42:45]

Gut. Da der uns von MonkeyLearn freundlicherweise zur Verfügung gestellte Account die Modellgrösse auf 3000 Samples limitiert, beschneiden wir unser Datenset auf 3250 Samples. Davon werden wir die ersten 3000 für das Training verwenden, und die folgenden 250 für die Validierung.

Verwende die Methode `sample()` des Python Moduls [random](https://docs.python.org/3.7/library/random.html), um die beiden Listen trump_tweets und other_tweets auf eine Länge von je 1625 Samples zu kürzen. So balancieren wir das Datenset auch gleich aus.

Neben der verlinkten Modulbeschreibung kannst das Python-interne Hilfesystem verwenden: Gib `help(random.sample)`. Manchmal funktioniert auch die Tab-Completion für Parameter nicht schlecht: `random.sample(<tab>...)`.

#### Aufgabe 1

In [ ]:
import random
trump_tweets = ...
other_tweets = ...

#### Vorschlag zur Umsetzung

In [ ]:
import random
trump_tweets = random.sample(trump_tweets, 1625)
other_tweets = random.sample(other_tweets, 1625)

OK, nun haben wir unsere Daten in Form von zwei Listen. Machen wir daraus einen Pandas DataFrame, damit können wir die Daten flexibel umformen (natürlich hätten wir auch gleich mit Pandas und mit `pd.read_json()` beginnen können).

Erstelle einen DataFrame mit einem Tweet pro Zeile und mit zwei Spalten, eine Spalte mit dem tweet Text aller Trump und NotTrump tweets und eine mit dem Label: für Trump-Tweets 'Trump' und für die anderen 'Other'.

#### Aufgabe 2

In [ ]:
# Erstelle den Pandas DataFrame wie oben beschrieben
import pandas as pd

df = pd.DataFrame(...)

df.columns=['text', 'tags'] # verwende diese zwei Spaltennamen
df = df.sample(frac=1) # mischen, dann können wir später einfach das Validierungsset hinten abschneiden
df.head()

#### Vorschlag zur Umsetzung

In [ ]:
import pandas as pd

# Es gibt viele Möglichkeiten, wie man diesen DataFrame erstellen kann.
# Hier eine kompakte, auch wenn vielleicht nicht die leserlichste
# Das .T am schluss dreht den DataFrame (.T für transpose)
df = pd.DataFrame([trump_tweets+other_tweets, ['Trump']*len(trump_tweets)+['Other']*len(other_tweets)]).T

df.columns=['text', 'tags'] # verwende diese zwei Spaltennamen
df = df.sample(frac=1) # mischen, dann können wir später einfach das Validierungsset hinten abschneiden
df.head()

Nun schauen wir uns das API von Monkeylearn an (via [offizieller Monkey Learn Python Client](https://github.com/monkeylearn/monkeylearn-python)). Hier ist die gesamte [API Referenz](https://monkeylearn.com/api/v3/#classifier-api).

Wir initialisieren das API und ertstellen ein Modell. Wähle im untenstehenden Code einen eigenen Classifier-Namen, möglichst so, dass die anderen Workshop-Teilnehmer nicht den gleichen Namen erwischen. Führe dann den Code aus.

In [ ]:
my_classifier_name = "..." # benenne hier Deinen classifier

In [ ]:
from monkeylearn import MonkeyLearn

API_KEY = ...

# Erstelle ein ml Objekt
ml = MonkeyLearn(API_KEY)

# Erstelle einen Klassifier. ACHTUNG: Mit dem zur Verfügung stehenden API Key
# können maximal 20 Classifier gleichzeitig erstellt werden!
res = ml.classifiers.create(my_classifier_name)
res.body

Mit dem Account, den wir verwenden, können genau 20 Classifiers erstellt werden. Bitte erstelle deshalb nur einen Classifier, damit die anderen Workshop-Teilnehmer auch einen machen können. Musst Du einen Classifier löschen, verwende das API oder melde Dich bei mir.

Schau Dir nun die oben verlinkte API Referenz an und vervollständige den folgenden Code:

#### Aufgabe 3

In [ ]:
# Hohl die ID des neuen Models
model_id = ...

# Erstelle zwei neue Tags (Klassen) mit den Bezeichnern 'Trump' und 'Other'
...
...

#### Vorschlag zur Umsetzung

In [ ]:
# Hohl die ID des neuen Models
model_id = res.body['id']

# Erstelle zwei neue Tags (Klassen) mit den Bezeichnern 'Trump' und 'Other'
res = ml.classifiers.tags.create(model_id, 'Trump')
res = ml.classifiers.tags.create(model_id, 'Other')

Als nächstes bereiten wir unsere Trainingsdaten vor.

Das Classifier API benötigt eine Liste von Samples, worin jedes Element ein Dictionary mit zwei Keys ist. Für den Key 'text' geben wir als Value den entsprechenden Tweet als String. Für den Key 'tags' geben wir als Value eine Liste mit Länge 1, deren Element das entsprechende Label als String ist, also entweder 'Trump' oder 'Other'.

Verwende nur die ersten 3000 Samples unseres Datarame: `df.iloc[0:3000]`, die restlichen 250 verwenden wir später zur Validierung.

#### Aufgabe 4

In [ ]:
# Erstelle eine Liste aus X_train und y_train

train_samples = ...

#### Vorschlag zur Umsetzung

In [ ]:
# Auch hier gibt es viele Wege, zum Ziel zu kommen. Einer davon, mit der in Python gängigen list comprehension:
train_samples = [{'text':d['text'], 'tags':[d['tags']]} for d in df.iloc[0:3000].to_dict(orient='records')]

Nun können wir die Trainingsdaten hochladen. MonkeyLearn started danach das Training automatisch.

#### Aufgabe 5

In [ ]:
# Samples uploaden
...

#### Vorschlag zur Umsetzung

In [ ]:
# Samples uploaden
ml.classifiers.upload_data(model_id, train_samples)

Das Training dauert nicht lange. Sobald der Classifier im Feld last_trained ein Datum enthält, wurde er trainiert. Frage dieses Feld ab, und wiedehole die Abfrage (durch mehrmaliges, manuelles Auusführen der Notebook-Zelle), bis das Training beendet wurde.

#### Aufgabe 6

In [ ]:
...

#### Vorschlag zur Umsetzung

In [ ]:
ml.classifiers.detail(model_id).body['last_trained']

Nun validieren wir das Modell mit unserem Validierungsset. Schaue in der API-Dokumentation nach, in welcher Form der API-Call `classsify()` die zu klassifizierenden Samples haben möchte, forme um und führe den Call aus.

**Achtung**: Da die verfügbaren Requests beschränkt sind, pass bitte auf, dass Du keine Endlosloops baust. Pro Teilnehmer stehen aber immerhin knapp 20'000 Requests für die Prediction zur Verfügung, ein bisschen Rumprobieren ist also erlaubt.

#### Aufgabe 7

In [ ]:
test_samples = df....

# Mache predictions für das ganze Validierungsset. Speichere sie vorerst so, wie sie von MonkeyLearn zurückkommen.
predictions = ...

#### Vorschlag zur Umsetzung

In [ ]:
test_samples = df.iloc[3000:].text.tolist()

# Mache predictions für das ganze Validierungsset. Speichere sie vorerst so, wie sie von Monkeylearn zurückkommen.
predictions = ml.classifiers.classify(model_id, test_samples).body

Ok, eine Prediction sieht so aus:

In [ ]:
predictions[0]

Das ist etwas unpraktisch, extrahieren wir das. Du kannst der Einfachheit halber davon ausgehen, dass hier immer genau ein Tag zurückgegeben wird.

#### Aufgabe 8

In [ ]:
pred = pd.DataFrame(...)

#### Vorschlag zur Umsetzung

In [ ]:
pred = pd.DataFrame([(d['text'], d['classifications'][0]['tag_name']) for d in predictions], columns=['text', 'tags'])

Wir kontrollieren noch kurz, ob die Reihenfolge unserer Testdaten mit derjenigen der Predictions immer noch übereinstimmt:

In [ ]:
df.iloc[3000:].head()

In [ ]:
pred.head()

Das scheint zu passen. Nun berechnen wir die Accuracy.

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(df.iloc[3000:].tags, pred.tags)

Nicht schlecht! Mehr als 80% der Samples wurden korrekt klassifiziert. Damit wären wir am Ende dieses Toy Projects.

Wenn Du möchtest, kannst nun etwas weiter experimentieren. Zum Beispiel einige Settings des Modells anpassen und neu trainieren und validieren:
 * Anstatt den Algorithmus Naive Bayes (nb) eine Support Vector Machine (svm) verwenden
 * max_features veroppeln
 * Unigrams und Bigrams verwenden (ngram_range -> [1, 2])
 * Unser primitives Preprocessing vom Anfang des Notebooks weglassen und dafür Preprocessing für Social Media einschalten
 * Stemming ausschalten
 
Oder wertest eine andere Metric als Accuracy aus, beispielsweise Area Under the ROC Curve oder Logarithmic Loss. Dazu benötigst Du nicht nur die Prediction, sondern auch die Confidence.

Aufschlussreich ist es beispielsweise auch, sich einmal das Dashboard von [Monkeylearn](https://monkeylearn.com) anzuschauen. Dazu müsstest Du Dich aber selber dort registrieren. Für den Free Tier braucht es lediglich eine Email-Adresse, damit kannst Du Modelle mit maximal 3000 Samples trainieren und pro Monat 1000 Requests machen. Trainieren zählt nicht zu den Requests.
Du kannst auch auf den Team Tier upgraden, dann bekommst Du 300'000 Requests. Dafür benötigst Du aber eine Kreditkarte, und nach 14 Tagen wird diese belastet, wenn Du vorher nicht kündigst.
Alternativ kannst Du das Dashboard und Deinen Klassifier auch kurz auf meinem Laptop anschauen.

Weitere Ideen:
* Ein paar neuere Trump tweets klassifizieren und schauen, ob unser Detektor diese auch als von Trump stammend erkennt. Tweets von 2017 sind im Image vorhanden.
* Tweets von jemand anderes als Trump und Ben Carson klassifizieren und schauen, ob unser Detektor diese auch als Nicht-Trump-Tweets erkennt
* Hillary Clinton hinzunehmen und zwischen allen dreien unterscheiden. Tip: Das Modell muss nicht neu erstellt werden, es reicht, nur die neuen Tweets von Hillary hinzuzufügen und nochmals zu trainieren. Dann aber überprüfen, ob bei den predictions nun jeweils mehr als ein Tag zurückkommt.

Und zuletzt noch dies: Wie ich vergangene Woche [zu lesen war](http://varianceexplained.org/r/trump-tweets/), stammen Trump Tweets nicht nur von Trump himself, sondern auch von seinem Staff. Es wäre interessant, diese Analyse nachzuvollziehen.

**Wichtig**: Wenn Du Deinen Classifier nicht mehr brauchst, so lösche ihn doch bitte gleich:

In [ ]:
ml.classifiers.delete(model_id)